<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Description" data-toc-modified-id="Description-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Description</a></span></li><li><span><a href="#Current-scenario" data-toc-modified-id="Current-scenario-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Current scenario</a></span></li><li><span><a href="#Load" data-toc-modified-id="Load-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load</a></span><ul class="toc-item"><li><span><a href="#Get-the-matrices" data-toc-modified-id="Get-the-matrices-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Get the matrices</a></span></li><li><span><a href="#Viz-costs" data-toc-modified-id="Viz-costs-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Viz costs</a></span></li></ul></li><li><span><a href="#Solve" data-toc-modified-id="Solve-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Solve</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Solving-the-linear-system" data-toc-modified-id="Solving-the-linear-system-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>Solving the linear system</a></span></li><li><span><a href="#SVD" data-toc-modified-id="SVD-4.0.2"><span class="toc-item-num">4.0.2&nbsp;&nbsp;</span>SVD</a></span></li></ul></li><li><span><a href="#Trying-to-&quot;trick&quot;-the-system-and-create-a-counter-example." data-toc-modified-id="Trying-to-&quot;trick&quot;-the-system-and-create-a-counter-example.-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Trying to "trick" the system and create a counter example.</a></span><ul class="toc-item"><li><span><a href="#Trying-to-see-if-the-algorithm-converges-or-not." data-toc-modified-id="Trying-to-see-if-the-algorithm-converges-or-not.-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Trying to see if the algorithm converges or not.</a></span></li><li><span><a href="#Analyze-the-results-and-the-associated-contractivity-ratio" data-toc-modified-id="Analyze-the-results-and-the-associated-contractivity-ratio-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Analyze the results and the associated contractivity ratio</a></span></li></ul></li></ul></li></ul></div>

In [ ]:
from matplotlib import rc
rc('text', usetex=True)
rc('font', size = 12)
rc('xtick', labelsize = 12)
rc('ytick', labelsize = 12)
rc('figure', figsize = (8, 4))

In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
from amod_ed.contractivity_v3 import viz_costs, _construct_problem
from amod_ed.contractivity_v3 import sample_solutions, run_algorithm
import cvxpy as cp
import networkx as nx

import pandas as pd

from amod_ed.contractivity_v3 import plot_results_run, compute_error_KKT, get_d_values

from amod_ed.contractivity_v3 import get_new_r, get_edge_flow, get_flow_edge_od

In [ ]:
import numpy as np

# Description

The previous notebook with the same name aimed at reproducing some scenarios that had been identified as potentially non contractive (svd >1 ) with Matlab. However, the way the 2D mapping was done was not correct. And therefore, it was ambiguous whether this svd>1 value should be taken seriously or not. 

Indeed, we might have: 
- in 3D a direction that could not really be explored
- in 2D the mapping was not properly defined I am afraid



# Current scenario

The scenario we want is the following: 

4     2     2     1     1     1

     1     1     0     1     0     0

# Load

We load and then edit the graph characteristics to make sure we match the Matlab analysis parameters. 

In [ ]:
edges = pd.read_excel('cost_edges_3.xlsx')
inv_edges = pd.read_excel('inv_demand_3.xlsx')

In [ ]:
edges['phi']=3

In [ ]:
edges['k']=1

In [ ]:
edges.loc[1:, 'phi'] = 50

In [ ]:
gamma = 40/3
edges.loc[0, 'phi'] = edges.loc[0, 'phi']*gamma
edges.loc[0, 'k'] = edges.loc[0, 'k']*gamma

In [ ]:
gamma = 3/50

edges.loc[2, 'phi'] = edges.loc[2, 'phi']*gamma*4
edges.loc[2, 'k'] = edges.loc[2, 'k']*gamma*4

In [ ]:
gamma = 50/500
edges.loc[3, 'phi'] = edges.loc[3, 'phi']*gamma
edges.loc[3, 'k'] = edges.loc[3, 'k']*gamma

In [ ]:
gamma = 3/50

edges.loc[5, 'phi'] = edges.loc[5, 'phi']*gamma
edges.loc[5, 'k'] = edges.loc[5, 'k']*gamma

In [ ]:
# edges.loc[3, 'phi'] = 140
edges.loc[3, 'phi'] = 50

In [ ]:
edges

In [ ]:
inv_edges['k'] = .5

In [ ]:
inv_edges['shift'] = 15

In [ ]:
inv_edges['phi']+=1

In [ ]:
inv_edges.loc[:, 'shift'] = 0
inv_edges.loc[0, 'shift'] = 44
inv_edges.loc[1, 'shift'] = 80
inv_edges.loc[3, 'shift'] = 35

In [ ]:
inv_edges

## Get the matrices

In [ ]:
alpha = 0.15

In [ ]:
edges['A'] = edges['phi']/edges['k']/2 * alpha

In [ ]:
edges

In [ ]:
#order of edges in matlab
#edges = [1,2;1,3;2,1;2,3;3,1;3,2];
new_order = [0, 2, 1, 4, 3, 5]
edges.loc[new_order, 'A']

In [ ]:
edges.loc[new_order, 'phi']

In [ ]:
inv_edges['D'] = inv_edges['phi']/inv_edges['k']/2 * alpha

In [ ]:
inv_edges.loc[new_order, 'D']

In [ ]:
inv_edges.loc[new_order, 'shift'] - inv_edges.loc[new_order, 'phi']

## Viz costs

Visualize the costs for each OD pair specified in the Excels. 

In [ ]:
# inv_edges['shift'] = inv_edges['shift']+15

In [ ]:
viz_costs(edges, inv_edges, name = 'costs1', save = False, correct = False, beta = 1)

# Solve

In [ ]:
correct = False

In [ ]:
"""
Returns
-------
f_p: cvxpy.Variable
    The flow for each commodity on each edge
f_r: cvxpy.Variable
    The rebalancing flow on each edge
r: cvxpy.Parameter
    The rebalancing guess for each node
d_var: cvxpy.Variable
    The demand for each each
prob: cvxpy.Problem
    The optimization problem
map_comps: dict
    A map linking components of f_p to the edges and inv edges
map_edges: dict
    A map linking edges to components of f_p
costs_dict: dict
    Dict containing the cost for each edge
inv_demand_dict: dict
    The inverse demand cost for each od pair
G: nx.DiGraph
    Graph representing the network
nodes: list
    list of nodes
"""
f_p, f_r, r, d_var, prob, map_comps, map_edges,\
 costs_dict, inv_d_dict, G, nodes = _construct_problem(edges, inv_edges, correct = correct, beta = 1)

Specify a value of the rebalancing parameter

In [ ]:
x = -1
y = 1


In [ ]:
r_new = [x, -x-y, y]
r.value = r_new

You can then solve the problem. 

In [ ]:
prob.solve(solver = cp.GUROBI)

Check the status. 

In [ ]:
prob.status

Therefore, the below function helps in analyzing and decomposing edge by edge for the passenger flow. 

In [ ]:
get_edge_flow(f_p, map_edges)

In [ ]:
get_d_values(inv_edges, d_var)

In [ ]:
edges

In [ ]:
f_r.value

In [ ]:
kkt_rel_error = compute_error_KKT(G, costs_dict, inv_d_dict, inv_edges, map_comps, f_p)

In [ ]:
kkt_rel_error

We can also compute directly the new rebalancing guess. 

In [ ]:
get_new_r(f_p, map_edges, nodes)

### Solving the linear system

We solve for 4 different values of $r$. We check that they all correspond to the same scenario. 

In [ ]:
y_vec = [.6, .75, .5, .8]
r_list = []
Tr_list = []
x_vec = [-.5, -1, -.75, -.6]
for i in range(4):
    x = x_vec[i]
    y = y_vec[i]
    r_new = [x, -x-y, y]
    r_list.append(r_new)
    r.value = r_new

    prob.solve(solver = cp.GUROBI)
    print("-------")
    print(get_edge_flow(f_p, map_edges))
    print(f_r.value)

    print(get_d_values(inv_edges, d_var))

    Tr_list.append(get_new_r(f_p, map_edges, nodes))

In [ ]:
#we construct the A matrix in order to find the corresponding mapping
b = np.zeros(4)
A = np.zeros((4,4))
for i in [0, 1]:
    dTr = np.array(Tr_list[i]) - np.array(Tr_list[i+1])
    dr = np.array(r_list[i]) - np.array(r_list[i+1])
    s = i*2
    drA = dr[:2]
    A[s, :2] = drA
    A[s+1, 2:] = drA
    b[s:s+2] = dTr[:2]

In [ ]:
A

In [ ]:
sol = np.linalg.solve(A, b)

In [ ]:
A_sol = [[sol[0], sol[1]], [sol[2], sol[3]]]

In [ ]:
np.around(A_sol, 4)


In [ ]:
#3D solution 

A_sol_3D = np.array([[1,0],[0, 1], [-1, -1]])@A_sol@np.array([[1, 0, 0], [0, 1, 0]])

In [ ]:
A_sol_3D

We check that $Tr_1 - Tr_2 = A(r_1 - r_2)$

In [ ]:
#test
i = 0
print(A_sol_3D@(np.array(r_list[i])- np.array(r_list[i+1])))
print(np.array(Tr_list[i]) - np.array(Tr_list[i+1]))

In [ ]:
x = np.array([1, -1, 0])/np.sqrt(2)
z = np.array([1, 1, 1])/np.sqrt(3)
y = np.cross(z, x)
T = np.array([x, y, z]).T
T = T[:, :2]

We map the solution matrix to 2D

In [ ]:
A_sol_2D = T.T@A_sol_3D@T

#This is what I am expecting to get: 

    0.8503    0.4967
    
   -0.0091    0.7675

In [ ]:
A_sol_2D

We see that the sv is >1. Also, we don't match with the one from matlab. 


Questions
1. why don't we match with that from Matlab

### SVD

We get the SVD of this 2D matrix

In [ ]:
u, s, vh = np.linalg.svd(A_sol_2D)

The first sv is larger than 1. 

In [ ]:
s

In [ ]:
A_sol_2D@vh[0,:]/s[0]

The direction corresponding to this sv is the first right singular vector. 

In [ ]:
np.linalg.norm(A_sol_2D@vh[0,:])

In [ ]:
vh[0,:]

## Trying to "trick" the system and create a counter example. 

We construct two rebalancing vectors, one based on this non-contractive direction. 

In [ ]:
r1 = r_list[0]

In [ ]:
#r in the 2D space
r1_ = T.T@r1

In [ ]:
dv = vh[0,:]

In [ ]:
alpha = 0.01

Here we construct $r_2 = r_1 +\alpha dv$

In [ ]:
r2_ = np.array(r1_)-alpha*np.array(dv)

We solve the problem with the solver (mapping the 2D to the 3D vectors). We check that both are still corresponding to the same scenario. 

In [ ]:
Tr_ = []
for r_new in [r1_, r2_]:
    rnew_3D = T@r_new
    r.value = rnew_3D

    prob.solve(solver = cp.GUROBI)
    print("-------")
    print(get_edge_flow(f_p, map_edges))
    print(f_r.value)

    print(get_d_values(inv_edges, d_var))

    Tr_.append(get_new_r(f_p, map_edges, nodes))

In [ ]:
dr = np.array(r1)-np.array(T@r2_)
dT = np.array(Tr_[0]) - np.array(Tr_[1])

dr_ = np.array(r1_)-np.array(r2_)
dT_ = np.array(T.T@Tr_[0]-T.T@Tr_[1])

Checking the norm of both the 3D and the 2D solutions. We see that both correspond. 

In [ ]:
print(np.linalg.norm(dr))
print(np.linalg.norm(dT))

In [ ]:
print(np.linalg.norm(dr_))

print(np.linalg.norm(dT_))

Checking the contractivity ratio. It is larger than 1. 

In [ ]:
np.linalg.norm(dT_)/np.linalg.norm(dr_)

In [ ]:
np.linalg.norm(dT)/np.linalg.norm(dr)

We also check that the solution matrix works for the chosen example (i.e. we remain in the same polyhedron). 

In [ ]:
A_sol_2D@dr_

In [ ]:
dT_

### Trying to see if the algorithm converges or not. 

In [ ]:
dr

In [ ]:
r0 = [dr, T@r1_, T@r2_]

In [ ]:
r_tot = run_algorithm(edges, inv_edges, r0 = r0, beta = 1, max_iter = 50)

In [ ]:
plot_results_run(r_tot)

### Analyze the results and the associated contractivity ratio

In [ ]:
r_ = r_tot[2]

In [ ]:
rat=[]
for i in range(len(r_k)-2):
    dr_crt = np.array(r_k[i+1])-np.array(r_k[i])
    dT_crt = np.array(r_k[i+2])-np.array(r_k[i+1])
    rat_crt = np.linalg.norm(dT_crt)/np.linalg.norm(dr_crt)
    rat.append(rat_crt)

In [ ]:
rat

In [ ]:
IMAGES_PATH = "/Users/lucasfuentes/ASL/Images/Contractivity_3Nodes/"

In [ ]:
import os

In [ ]:
_, ax = plt.subplots(2, 1, figsize = (8,4))
for r_k in r_tot:
    r_arr = np.array(r_k)
    diff = [np.linalg.norm(r_arr[i,:] - r_arr[i+1,:]) for i in range(r_arr.shape[0]-1)]
    ax[0].plot(diff)
    rat=[]
    for i in range(len(r_k)-2):
        dr_crt = np.array(r_k[i+1])-np.array(r_k[i])
        dT_crt = np.array(r_k[i+2])-np.array(r_k[i+1])
        rat_crt = np.linalg.norm(dT_crt)/np.linalg.norm(dr_crt)
        rat.append(rat_crt)
    ax[1].plot(rat)
ax[0].grid()
ax[1].grid()
ax[0].set_xlabel('$k$')
ax[0].set_ylabel('$\|r_k-r_{k+1}\|$')
ax[0].set_yscale('log')
ax[1].set_xlabel('$k$')
ax[1].set_ylabel('$\|sdf{k+1}\|/|r_k-r_{k+1}\|$')
plt.savefig(os.path.join(IMAGES_PATH, "non_contractivity.png"),dpi = 200, transparent =True)

In [ ]:
_, ax = plt.subplots(2, 1, figsize = (8,4))
for r_k in r_tot:
    r_arr = np.array(r_k)
    diff = [np.linalg.norm(r_arr[i,:] - r_arr[i+1,:]) for i in range(r_arr.shape[0]-1)]
    ax[0].plot(diff)
    rat=[]
    for i in range(len(r_k)-2):
        dr_crt = np.array(r_k[i+1])-np.array(r_k[i])
        dT_crt = np.array(r_k[i+2])-np.array(r_k[i+1])
        rat_crt = np.linalg.norm(dT_crt)/np.linalg.norm(dr_crt)
        rat.append(rat_crt)
    ax[1].plot(rat)
ax[0].grid()
ax[1].grid()
ax[0].set_xlabel('$k$')
ax[0].set_ylabel('$\|r_k-r_{k+1}\|$')
ax[0].set_yscale('log')
ax[1].set_xlabel('$k$')
ax[1].set_ylabel('$\|T_k - T_{k_1}\|/\|r_k-r_{k+1}\|$')
plt.savefig(os.path.join(IMAGES_PATH, "non_contractivity.png"),dpi = 200, transparent =True)